In [1]:
import numpy as np, matplotlib.pyplot as plt
import mxnet as mx
import os
import cv2

In [2]:
rec_path = os.path.join('..', 'data', 'datasets', 'face recognition', 'VGG-Face', 'faces_vgg_112x112', 'train.rec')
idx_path = os.path.join('..', 'data', 'datasets', 'face recognition', 'VGG-Face', 'faces_vgg_112x112', 'train.idx')
output_dir = os.path.join('..', 'data', 'datasets', 'face recognition', 'VGG-Face', 'aligned')

os.makedirs(output_dir, exist_ok=True)

record = mx.recordio.MXIndexedRecordIO(idx_path, rec_path, 'r')

In [3]:


for i, key in enumerate(record.keys):
    try:
        header, img = mx.recordio.unpack(record.read_idx(key))
        if isinstance(header.label, list) or isinstance(header.label, np.ndarray):
            label = int(header.label[0])
        else:
            label = int(header.label)
            
        img_arr = cv2.cvtColor(mx.image.imdecode(img).asnumpy(), cv2.COLOR_BGR2RGB)  # Decode to ndarray
        save_dir = os.path.join(output_dir, str(label))
        os.makedirs(save_dir, exist_ok=True)
        
        
        cv2.imwrite(os.path.join(save_dir, f"{i}.jpg"), img_arr)

        if i % 10000 == 0:
            print(f"Processed {i}/{len(record.keys)}")
    except:
        print(i, label)
        continue

Processed 0/3146439
Processed 10000/3146439
Processed 20000/3146439
Processed 30000/3146439
Processed 40000/3146439
Processed 50000/3146439
Processed 60000/3146439
Processed 70000/3146439
Processed 80000/3146439
Processed 90000/3146439
Processed 100000/3146439
Processed 110000/3146439
Processed 120000/3146439
Processed 130000/3146439
Processed 140000/3146439
Processed 150000/3146439
Processed 160000/3146439
Processed 170000/3146439
Processed 180000/3146439
Processed 190000/3146439
Processed 200000/3146439
Processed 210000/3146439
Processed 220000/3146439
Processed 230000/3146439
Processed 240000/3146439
Processed 250000/3146439
Processed 260000/3146439
Processed 270000/3146439
Processed 280000/3146439
Processed 290000/3146439
Processed 300000/3146439
Processed 310000/3146439
Processed 320000/3146439
Processed 330000/3146439
Processed 340000/3146439
Processed 350000/3146439
Processed 360000/3146439
Processed 370000/3146439
Processed 380000/3146439
Processed 390000/3146439
Processed 4000

In [4]:
output_dir = os.path.join('..', 'data', 'datasets', 'face recognition', 'VGG-Face', 'aligned')

In [ ]:
images = 0
identities = os.listdir(output_dir).__len__()
mini = 100000000
maxi = 0
for directory, dirnames, filenames in os.walk(output_dir):
    images += filenames.__len__()


['820', '791', '4943', '7425', '5732', '5398', '4611', '5285', '2489', '3429', '5011', '724', '7840', '8364', '7399', '5650', '100', '3270', '5347', '2174', '6485', '3541', '2348', '5876', '8269', '4081', '7213', '4084', '8533', '7828', '5043', '5323', '6602', '4683', '6420', '46', '1460', '8010', '8525', '6111', '1289', '8621', '3946', '6523', '1763', '5110', '6812', '3037', '2623', '2127', '7757', '7157', '5281', '7117', '2862', '6122', '1722', '7732', '2510', '6640', '6815', '6446', '6004', '8108', '2524', '7491', '779', '2552', '744', '3324', '7167', '4101', '8023', '5', '6278', '6041', '496', '8049', '472', '7744', '7227', '8343', '1828', '7360', '7644', '5829', '3485', '7735', '514', '5270', '2954', '6691', '5314', '695', '8361', '2336', '4630', '7490', '4062', '6788', '4962', '4977', '5034', '2333', '912', '4569', '3433', '1667', '5619', '4528', '4822', '5812', '8332', '4421', '237', '4625', '5009', '858', '1940', '5795', '6858', '3812', '7758', '4514', '6603', '5749', '8399', '

In [14]:
images, identities

(3137807, 8631)